## Class

In [2]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        

## Load Data

In [3]:
import json

file_name = '/Users/Ernesto/Python_Data_Science/Amazon data/sklearn-master/data/sentiment/Books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

In [4]:
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## Prep Data

In [29]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

len(train_container.reviews)

6700

In [30]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

In [31]:
print(test_y.count(Sentiment.POSITIVE))
print(test_y.count(Sentiment.NEGATIVE))

208
208


## Bag of words vectorization

In [38]:
# As some words are frequently used in a text and carry very little information, we can use TfidfVectorizer
# in order to re-weight the count features into floating point values suitable for usage by a classifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())


This is really great, i have been doing this in a very small way for several years and now i can expand to so many other great water infusions. Since i am a diabetic this will give me so many options. i am very excited to begin today.
[[0. 0. 0. ... 0. 0. 0.]]


## Classification

#### Linear SVM

In [39]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

'A good read about a very real problem.  If you are a caregiver for a person with a disability you might not care for this book.'

In [40]:
clf_svm.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Decision Tree

In [41]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [42]:
clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Naive Bayes

In [43]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(), train_y)

GaussianNB(priors=None)

In [44]:
clf_gnb.predict(test_x_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

#### Logistic Regression

In [45]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [46]:
clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

## Evaluation

### Accuracy

In [47]:
print("SVM mean accuracy = " + str(clf_svm.score(test_x_vectors, test_y)))
print("Decision tree mean accuracy = " + str(clf_dec.score(test_x_vectors, test_y)))
print("Naive Bayes mean accuracy = " + str(clf_gnb.score(test_x_vectors.toarray(), test_y)))
print("Logistic Regression mean accuracy = " + str(clf_log.score(test_x_vectors, test_y)))

SVM mean accuracy = 0.8076923076923077
Decision tree mean accuracy = 0.6610576923076923
Naive Bayes mean accuracy = 0.6610576923076923
Logistic Regression mean accuracy = 0.8028846153846154


### F1 Score

In [48]:
from sklearn.metrics import f1_score

print("F1 Score for SVM = " + str(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])))
#print("F1 Score for Dec Tree = " + str(f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])))
#print("F1 Score for Naive Bayes = " + str(f1_score(test_y, clf_gnb.predict(test_x_vectors.toarray()), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])))
#print("F1 Score for Log Reg = " + str(f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])))

F1 Score for SVM = [0.80582524 0.80952381]


In [49]:
test_set = ['very good book', 'bad book do not buy', "horrible waste of time"]
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

### Tuning our model (with Grid Search)

In [50]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ('linear', 'rbf'), 'C': (1, 4, 8, 16, 32)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [51]:
print("SVM mean accuracy = " + str(clf.score(test_x_vectors, test_y)))

SVM mean accuracy = 0.8052884615384616


###  Saving Model

In [54]:
import pickle

with open('/Users/Ernesto/Python_Data_Science/Amazon data/models_Ernesto/Amazon_Sentiment_Classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

### Load the model

In [55]:
with open('/Users/Ernesto/Python_Data_Science/Amazon data/models_Ernesto/Amazon_Sentiment_Classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [61]:
print(test_x[5])
loaded_clf.predict(test_x_vectors[5])

I really liked this book. It's not your typical kind of romance novel. It's down-to-earth and funny and tells a story that could actually happen. Cindy is a smart woman who never gives up and learns from her mistakes. This is why she's ultimately able to enter into a relationship that's good for her, and find love on equal terms. But there's wisdom in here for anyone who wants to change their life for the better. I'm off to buy The Sugar Ticket to see what happens to Cindy next!


array(['POSITIVE'], dtype='<U8')